In [2]:
"""
Run live prediction model
"""

# Import packages
import numpy as np
import pandas as pd
import sys
from datetime import datetime, timedelta
import time
sys.path.append('../..')
import pickle
import krakenex
from pykrakenapi import KrakenAPI

import h5py
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, MaxPooling2D, Conv2D
from keras.callbacks import ModelCheckpoint, Callback
import matplotlib.pyplot as plt
import missingno as msno
import config

from Classifier.data_processing import processor
from Classifier.prediction_model import LSTM_net

def trade_script(Date):
    # Define global variables
    interval = 1440 # 1440 minutes = 1 day
    sequence_length = 4
    input_size = 35
    learning_rate = 0.00001 # Only needed to define optimiser. Not used in prediction
    input_size = (sequence_length, input_size)

    # Initialise model
    LSTM_network = LSTM_net(input_size, learning_rate)

    # Load the model weights with the best validation loss.
    LSTM_network.model.load_weights('/Users/alastairong/Documents/GitHub/traderbot/saved_models/LSTM_weights.hdf5')

    live_trading = False

    # Load normalisation data
    with open('/Users/alastairong/Documents/GitHub/traderbot/pickles/normalisation.pickle', 'rb') as f:
        x_mean, x_std, y_mean, y_std = pickle.load(f)

    # Load latest trade positions from log
    try:
        log = pd.read_csv('trade_log.csv')
        cash = log[-1]['Cash_Position']
        position = log[-1]['BTC_Position']
        print("loaded")
    except:
        cash = 1000
        position = 0
        print(cash)
        print(position)

    # Run prediction script
    input_data, current_price = processor.live_download(interval, sequence_length, x_mean, x_std)
    raw_prediction = LSTM_network.model.predict(input_data)
    expected_growth = raw_prediction.item() * y_std + y_mean
    predicted_price = current_price * (1 + expected_growth)

    # Simple trading algorithm.
    if expected_growth > 0:
        action = "buy"
        position += cash / current_price * 0.997 # 0.997 to account for fees
        volume = position
        cash = 0
    if expected_growth < 0:
        action = "sell"
        cash += position * current_price * 0.997
        volume = position
        position = 0

    if live_trading:
        # call Kraken API to make trades here
        api = krakenex.API(config.key, config.secret)
        k = KrakenAPI(api)
        k.add_standard_order('XXBTZUSD', action, 'market', volume)

    # Reporting and logging
    log_data = np.array([Date, action, current_price, predicted_price, cash, position, (cash + position * current_price)]).reshape(-1,7)
    log = pd.DataFrame(log_data, columns=["Date", "Action", "Current Price", "Predicted_Price", "Cash_Position", "BTC_Position", "Portfolio_Value"])
    log.to_csv('/Users/alastairong/Documents/GitHub/traderbot/test_log.csv', encoding='utf-8', index=True)

    print("{}: {}. Price expected to change from {} to {}. Portfolio value of {}".format(Date, action, current_price, predicted_price, (current_price * position + cash)))


Using TensorFlow backend.


In [3]:
start_date = datetime(2018, 2, 25) 
day_count = 22

for date in (start_date + timedelta(n) for n in range(day_count)):
    print(date)

2018-02-25 00:00:00
2018-02-26 00:00:00
2018-02-27 00:00:00
2018-02-28 00:00:00
2018-03-01 00:00:00
2018-03-02 00:00:00
2018-03-03 00:00:00
2018-03-04 00:00:00
2018-03-05 00:00:00
2018-03-06 00:00:00
2018-03-07 00:00:00
2018-03-08 00:00:00
2018-03-09 00:00:00
2018-03-10 00:00:00
2018-03-11 00:00:00
2018-03-12 00:00:00
2018-03-13 00:00:00
2018-03-14 00:00:00
2018-03-15 00:00:00
2018-03-16 00:00:00
2018-03-17 00:00:00
2018-03-18 00:00:00


In [33]:
def date_to_iso8601(date):
    return '{year}-{month:02d}-{day:02d}T{hour:02d}:{minute:02d}:{second:02d}'.format(
      year=date.year,
      month=date.month,
      day=date.day,
      hour=date.hour,
      minute=date.minute,
      second=date.second)

url = 'https://api.gdax.com/products/BTC-USD/candles'
start = datetime(2018, 3, 18)
end = datetime(2018, 3, 19)

In [34]:
iso_start = date_to_iso8601(start)
iso_end = date_to_iso8601(end)
import requests


response = requests.get(url, {
  'start': iso_start,
  'end': iso_end,
  'granularity': 1440 * 60 # Converting to seconds for API
})

In [35]:
print(response.json())

[[1521331200, 7310, 8302.73, 7857.59, 8192, 30526.278569373226]]


In [36]:
import numpy as np
import pandas as pd
import time
import requests
from datetime import datetime, timedelta
import krakenex
import pytz
from pykrakenapi import KrakenAPI
api = krakenex.API()
k = KrakenAPI(api)

In [37]:
interval = 1440

def date_to_interval(dt, interval):
    rounding = interval // 60
    hourly = dt.hour // rounding * rounding
    time_period = datetime(dt.year, dt.month, dt.day, hourly, 0)
    return time_period

def to_ohlc(trades):
    """
    Groups and processes individual trade data to return OHLC (candle) data
    :trades: trades pandas dataframe as returned by request_trade_slice function
    Returns an array with rows of candlestick data in the following format: ['time', 'low', 'high', 'open', 'close', 'volume']
    """
    # Converts unix timestamp data into candle interval periods. Period time corresponds to beginning of period
    trades['datetime'] = pd.to_datetime(trades['time'], unit='s') # Reformat unix timestamp as datetime
    trades['period'] = trades['datetime'].map(lambda x: date_to_interval(x, interval))

    # Group trades into periods to aggregate volume and get low/high price
    trade_agg = trades.groupby('period')
    trade_agg = trade_agg.agg({
        'price': {'low': np.min, 'high': np.max},
        'volume': np.sum
    })

    # Create a fresh pandas dataframe and copies data from aggregated trades dataframe above
    ohlc = pd.DataFrame(index=trade_agg.index.values, columns=['low', 'high', 'open', 'close', 'volume'])
    ohlc['low'] = trade_agg['price']['low']
    ohlc['high'] = trade_agg['price']['high']
    ohlc['volume'] = trade_agg['volume']['sum']

    # Iterate through each candle period and searches original trades dataframe for the first and last price, then set that
    for i, row in ohlc.iterrows():
        selection = trades.loc[trades['period'] == i] # Return all rows for the current period
        first_trade_index = selection['time'].idxmax()
        first_trade_price = selection.loc[first_trade_index]['price']
        ohlc.at[i, 'open'] = first_trade_price
        last_trade_index = selection['time'].idxmin()
        last_trade_price = selection.loc[last_trade_index]['price']
        ohlc.at[i, 'close'] = last_trade_price

    return ohlc


In [52]:
def request_trade_slice(start):
    """
    Calls krakenex get_trade function to get line-by-line trade data for an individual time slice
    :start: start of API request time period
    Returns a tuple of (trades, last). Last = last trade timestamp. Used to set timestamp for next request
    trades is a list of trades with format of [time, price, volume]
    """
    timestamp = int(start.replace(tzinfo=pytz.utc).timestamp()) * 1000000000
    trades, last = k.get_recent_trades('XXBTZUSD', since=timestamp, ascending=True)
    return trades, last/1000000000

slice_start = start
end_timestamp = end.replace(tzinfo=pytz.utc).timestamp() # last is returned a an epoch timestamp so end_time needs to be reformatted
trades, last = request_trade_slice(slice_start)
while last < end_timestamp:
    slice_start = datetime.utcfromtimestamp(last)
    new_data, last = request_trade_slice(slice_start)
    trades = trades.append(new_data)
    time.sleep(1)
    print("time period from {} to {}".format(slice_start, last))
trades = trades[trades.index < end]

dataframe = to_ohlc(trades)

time period from 2018-03-18 00:35:03.305289 to 1521334513.5536854
time period from 2018-03-18 00:55:13.553685 to 1521335627.7498422
time period from 2018-03-18 01:13:47.749842 to 1521338477.9300518
time period from 2018-03-18 02:01:17.930052 to 1521339063.516527
time period from 2018-03-18 02:11:03.516527 to 1521340495.2168415
time period from 2018-03-18 02:34:55.216841 to 1521341836.4929605
time period from 2018-03-18 02:57:16.492960 to 1521343098.3564706
time period from 2018-03-18 03:18:18.356471 to 1521345494.816861
time period from 2018-03-18 03:58:14.816861 to 1521347623.5875673
time period from 2018-03-18 04:33:43.587567 to 1521348869.7121477
time period from 2018-03-18 04:54:29.712148 to 1521349596.6637979
time period from 2018-03-18 05:06:36.663798 to 1521352926.0586061
time period from 2018-03-18 06:02:06.058606 to 1521355415.0341406
time period from 2018-03-18 06:43:35.034141 to 1521359511.615142
call rate limiter exceeded (counter=20, limit=20) 
 sleeping for 5 seconds
time

/Users/alastairong/anaconda3/lib/python3.6/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [49]:
dataframe

,low,high,open,close,volume
2018-03-18,7305.0,8249.1,8183.2,7839.1,18729.271286
2018-03-19,8183.2,8295.1,8282,8183.2,61.551594


In [50]:
trades = trades[trades.index < end]

In [51]:
print(trades.tail())

                             price    volume          time buy_sell  \
dtime                                                                 
2018-03-18 23:59:40.953000  8193.0  0.044606  1.521418e+09      buy   
2018-03-18 23:59:40.957200  8193.0  0.094191  1.521418e+09      buy   
2018-03-18 23:59:40.960900  8193.1  0.326727  1.521418e+09      buy   
2018-03-18 23:59:46.955000  8183.2  0.115000  1.521418e+09     sell   
2018-03-18 23:59:46.960100  8183.2  0.015400  1.521418e+09     sell   

                           market_limit misc                   datetime  \
dtime                                                                     
2018-03-18 23:59:40.953000        limit      2018-03-18 23:59:40.953000   
2018-03-18 23:59:40.957200        limit      2018-03-18 23:59:40.957200   
2018-03-18 23:59:40.960900        limit      2018-03-18 23:59:40.960900   
2018-03-18 23:59:46.955000        limit      2018-03-18 23:59:46.955000   
2018-03-18 23:59:46.960100        limit      2018-03